<a href="https://colab.research.google.com/github/Mahmood-Anaam/vinvl_bert/blob/last/notebooks/vinvl_bert_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vinvl Bert Demo







In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


---

### Installation

In [2]:
package_name = "vinvl-0.1.0-cp310-cp310-linux_x86_64.whl"
!pip install https://github.com/Mahmood-Anaam/vinvl/raw/main/{package_name} --quiet
!pip install git+https://github.com/Mahmood-Anaam/vinvl_bert.git --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 56.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.6/473.6 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!pip install datasets --quiet

### Import all libraries

In [5]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import torch
from PIL import Image
from vinvl_bert.feature_extractors import VinVLFeatureExtractor
from vinvl_bert.pipelines import VinVLBertPipeline
from vinvl_bert.configs import VinVLBertConfig

### Configuration

In [6]:
import torch
from vinvl_bert.configs import VinVLBertConfig


cfg = VinVLBertConfig


# General settings
cfg.model_id = "jontooy/AraBERT32-Flickr8k"  # model id from huggingface
cfg.device = "cuda" if torch.cuda.is_available() else "cpu"  # Device for computation (GPU/CPU)

# Dataset settings
cfg.dataset_path = "MahmoodAnaam/vqav2-ar-en-validation-2"  # Path or name of the dataset
cfg.language = "ar"  # Language for questions/answers ("ar" for Arabic, "en" for English)
cfg.split = "validation"  # Dataset split to use ("train", "validation", "test")
cfg.save_shard_dir = "/content/drive/MyDrive/VQAv2_Features_Captions" # Directory to save extracted features

cfg.num_shards = 10  #How many shards to split the dataset into.
cfg.current_index_shard = 0  # Which shard to select and return
cfg.batch_size = 30  # Number of samples per batch
cfg.username = "MahmoodAnaam"  # Username for Hugging Face


# Image and object detection settings
cfg.add_od_labels = True  # Whether to add object detection labels to input
cfg.max_img_seq_length = 50  # Maximum sequence length for image features

# Text input settings
cfg.max_seq_length = 70  # Maximum sequence length for text input
cfg.max_seq_a_length = 40  # Maximum sequence length for primary text (e.g., question)
cfg.is_train = False  # Whether the configuration is for training or inference
cfg.mask_prob = 0.15  # Probability of masking tokens during training
cfg.max_masked_tokens = 3  # Maximum number of tokens to mask in a single sequence

# Generation settings
cfg.is_decode = True  # Enable decoding (generation mode)
cfg.do_sample = False  # Whether to use sampling for generation
cfg.max_gen_length = 50  # Maximum length for generated text
cfg.num_beams = 5  # Number of beams for beam search
cfg.temperature = 1.0  # Temperature for sampling (lower values make output more deterministic)
cfg.top_k = 50  # Top-k sampling (0 disables it)
cfg.top_p = 1.0  # Top-p (nucleus) sampling (0 disables it)
cfg.repetition_penalty = 1.0  # Penalty for repeating words (1.0 disables it)
cfg.length_penalty = 1.0  # Penalty for sequence length (used in beam search)
cfg.num_keep_best = 3  # Number of best sequences to keep


In [7]:
import torch

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)

PyTorch version: 2.5.1+cu121
CUDA available: True
CUDA version: 12.1


### Dataset

In [8]:
from huggingface_hub import login
from datasets import load_dataset
from google.colab import userdata

In [9]:
# Login to Hugging Face
HF_TOKEN = userdata.get('HF_TOKEN')
login(token=HF_TOKEN)

In [10]:
dataset = load_dataset(cfg.dataset_path,split=cfg.split,trust_remote_code=True)
dataset

vqav2-ar-en-validation-2.py:   0%|          | 0.00/8.15k [00:00<?, ?B/s]

(…)nEnded_mscoco_val2014_questions.json.zip:   0%|          | 0.00/6.47M [00:00<?, ?B/s]

v2_mscoco_val2014_annotations.json.zip:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['metadata', 'image', 'question', 'answers', 'multiple_choice_answer'],
    num_rows: 214354
})

In [11]:
ds_shard = dataset.shard(num_shards=cfg.num_shards,index=cfg.current_index_shard)
ds_shard

Dataset({
    features: ['metadata', 'image', 'question', 'answers', 'multiple_choice_answer'],
    num_rows: 21436
})

In [12]:
ds_shard[0]

{'metadata': {'image_id': 262148,
  'question_id': 262148000,
  'question_type': 'none of the above',
  'answer_type': 'other'},
 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x512>,
 'question': {'en': 'Where is he looking?', 'ar': 'أين ينظر؟'},
 'answers': {'en': ['down',
   'down',
   'at table',
   'skateboard',
   'down',
   'table',
   'down',
   'down',
   'down',
   'down'],
  'ar': ['تحت',
   'تحت',
   'على الطاولة',
   'لوح التزلج',
   'تحت',
   'طاولة',
   'تحت',
   'تحت',
   'تحت',
   'تحت'],
  'confidence': ['yes',
   'yes',
   'yes',
   'yes',
   'yes',
   'yes',
   'yes',
   'yes',
   'yes',
   'yes'],
  'id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]},
 'multiple_choice_answer': {'en': 'down', 'ar': 'تحت'}}

## Extract Features

In [14]:
from vinvl_bert.feature_extractors import VinVLFeatureExtractor
from vinvl_bert.pipelines.vinvl_bert_pipeline import VinVLBertPipeline

In [15]:
feature_extractor = VinVLFeatureExtractor(device=cfg.device,add_od_labels=cfg.add_od_labels)

In [16]:
feature_extractor.model.device

'cuda'

In [17]:
pipeline = VinVLBertPipeline(cfg)

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/765 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/761k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/550M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/2.29k [00:00<?, ?B/s]

In [20]:
pipeline.feature_extractor.model.device,pipeline.cfg.device

('cuda', 'cuda')

In [21]:
def extract_features(examples):
  try:
    size = len(examples)
    examples["features"] = feature_extractor(examples["image"])
  except Exception as e:
    examples["features"] = [None]*size
    print(f"Error while extracting features: {e}")
  return examples


# ..................................................

def features_captions(examples):
  try:
    size = len(examples)
    features,captions = pipeline(examples["image"])
    examples["features"] = features
    examples["captions"] = captions
  except Exception as e:
    examples["captions"] = [None] * size
    examples["features"]  = [None] * size
    print(f"Error while extracting features: {e}")
  return examples

# ..................................................




ds  = ds_shard.map(features_captions,
                  batched=True,
                  batch_size=cfg.batch_size,
                  desc="Extracting Features & Captions",
                  )

shard_dir = f"{cfg.save_shard_dir}_{cfg.current_index_shard}"
ds.save_to_disk(shard_dir)

Extracting Features & Captions:   0%|          | 0/4 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4 [00:00<?, ? examples/s]

In [22]:
ds[0].keys()

dict_keys(['metadata', 'image', 'question', 'answers', 'multiple_choice_answer', 'features', 'captions'])

In [23]:
ds[0]['features'].keys()

dict_keys(['boxes', 'classes', 'img_feats', 'od_labels', 'scores', 'spatial_features'])

In [24]:
ds[0]['captions'][0].keys()

dict_keys(['caption', 'confidence'])

In [25]:
ds[0]['captions']

[{'caption': 'رجل يقوم بخدعة على لوح التزلج الخاص به',
  'confidence': 0.5949516892433167},
 {'caption': 'رجل يقوم بخدعة لوح التزلج الخاص به',
  'confidence': 0.5577831864356995},
 {'caption': 'رجل يقوم بخدعة على لوح التزلج الخاص به امام حشد من الناس',
  'confidence': 0.5361678600311279}]



---



### Merge Dataset Segments

In [ ]:
from datasets import load_from_disk

shards_paths = [ f"{cfg.save_shard_dir}_{index}" for index in range(cfg.num_shards)]
datasets = [load_from_disk(path) for path in shards_paths]
datasets

[Dataset({
     features: ['metadata', 'image', 'question', 'answers', 'multiple_choice_answer', 'features', 'captions'],
     num_rows: 4
 })]

In [ ]:
from datasets import concatenate_datasets

merged_dataset = concatenate_datasets(datasets,split=cfg.split)
merged_dataset

Dataset({
    features: ['metadata', 'image', 'question', 'answers', 'multiple_choice_answer', 'features', 'captions'],
    num_rows: 4
})

In [ ]:
from huggingface_hub import Repository, create_repo

# Create a repo on Hugging Face Hub
repo_id=f"{cfg.username}/{cfg.save_shard_dir.split('/')[-1].split('_')[0]}-VinVL-BiT-Captions"
repo_url = create_repo(repo_id=repo_id, private=True, exist_ok=True,repo_type="dataset")
print(f"Dataset repository URL: {repo_url}")

Dataset repository URL: https://huggingface.co/datasets/MahmoodAnaam/VQAv2-VinVL-BiT-Captions


In [ ]:
# Push the dataset to the repo
merged_dataset.push_to_hub(repo_id=repo_id)
print(f"Dataset pushed to {repo_id}")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Dataset pushed to MahmoodAnaam/VQAv2-VinVL-BiT-Captions


In [ ]:
from datasets import load_dataset

ds = load_dataset(repo_id)
ds

README.md:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/4 [00:00<?, ? examples/s]

DatasetDict({
    validation: Dataset({
        features: ['metadata', 'image', 'question', 'answers', 'multiple_choice_answer', 'features', 'captions'],
        num_rows: 4
    })
})